In [1]:
# Import all the necessary libraries
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPool2D
import keras
from keras import callbacks
from keras.layers import Dense, Dropout, Add, Input, BatchNormalization, Activation
from keras.optimizers import adam_v2

import matplotlib.pyplot as plt

import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
###################################################################################
# DATA AUGMENTATION --> this helps to expose the model to more aspects of the data
# original images - RGB coefficients in the 0-255, but they are too high for our model to process
# we target values between 0 and 1 instead by scaling with a 1/255
train_data_gen = ImageDataGenerator(rescale=1 / 255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     rotation_range = 15,
                                     fill_mode = 'reflect',
                                     data_format = 'channels_last',
                                     brightness_range = [0.5, 1.5]
                                    )

# No modifications are made in the test set, except for rescaling
val_data_gen = ImageDataGenerator(rescale=1 / 255)


###################################################################################
# Choose the batch size of train and validation, and also the target size of the images
train_batch_size = 32
val_batch_size = 32
target_size = (250, 250)

training_set = train_data_gen.flow_from_directory('weather/data_train',
                                                  target_size=(target_size),
                                                  batch_size=train_batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

validation_set = val_data_gen.flow_from_directory('weather/data_val',
                                                  target_size=target_size,
                                                  batch_size=val_batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 786 images belonging to 4 classes.
Found 167 images belonging to 4 classes.


In [9]:
###################################################################################
# Callback´s
def callbacks_list():
    lr = callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=5, min_lr=1e-30, cooldown=3,
                                     verbose=1)  # ?????????????????????

    file_path = 'modelos'
    cp = callbacks.ModelCheckpoint(file_path, monitor='accuracy', verbose=1, save_best_only=True,
                                   mode='auto')  # ????????????????????????????
    es = callbacks.EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=10)  # ????????????????????????

    return [lr]


In [10]:
###################################################################################
# Plot training & validation accuracy values
def plot_accuracy(hist):
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()


# Plot training & validation loss values
def plot_loss(hist):
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()
    

# def check_overfit(hist):
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     ax1.plot_accuracy(hist)
#     ax2.plot_loss(hist)

In [ ]:
def classification_matrix():
    
    
    

# MODEL 1


In [41]:
#Initializing the he_uniform kernel for weights
initializer=keras.initializers.he_uniform(seed=200)

#Sequential Model Building
model = Sequential()
                                          


#BLOCK 1
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'same',input_shape = (250,250,3), kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2,2)))

#BLOCK 2
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#BLOCK 3
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#BLOCK 4 
model.add(Conv2D(filters = 256, kernel_size = (5,5),padding = 'same',kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(Conv2D(filters = 256, kernel_size = (5,5),padding = 'same',kernel_initializer=initializer))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#Classifier
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation = "relu",kernel_initializer=initializer))
model.add(Dropout(0.5))
model.add(Dense(128, activation = "relu",kernel_initializer=initializer))
model.add(Dropout(0.5))
model.add(Dense(4, activation = 'softmax'))


#Using Adam optimizer with an initial learning rate of 0.0001
opt=adam_v2.Adam(lr=0.001, beta_1=0.91, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Compile the model
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 250, 250, 32)      896       
_________________________________________________________________
batch_normalization_50 (Batc (None, 250, 250, 32)      1000      
_________________________________________________________________
activation_50 (Activation)   (None, 250, 250, 32)      0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 250, 250, 32)      9248      
_________________________________________________________________
activation_51 (Activation)   (None, 250, 250, 32)      0         
_________________________________________________________________
batch_normalization_51 (Batc (None, 250, 250, 32)      1000      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 125, 125, 32)    

In [13]:
model_1 = Sequential()

# BLOCK 1
model_1.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(200, 200, 3)))
model_1.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))

# BLOCK 2
model_1.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model_1.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# BLOCK 3
model_1.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model_1.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))



# Classifier
model_1.add(GlobalMaxPool2D())
model_1.add(Dense(512, activation="relu"))
model_1.add(Dropout(0.5))
model_1.add(Dense(4, activation='softmax'))

# Compiling the CNN
# Choose the right optimizer ???????????????????
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 50, 50, 128)      

In [ ]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [14]:
###################################################################################
# Fit the model
hist_model_1 = model_1.fit(training_set,
                           steps_per_epoch=training_set.n // 32,
                           epochs=50,
                           validation_data=validation_set,
                           validation_steps=validation_set.n // 32,
                           callbacks=callbacks_list()
                          )

plot_accuracy(hist_model_1)
plot_loss(hist_model_1)


Epoch 1/50
 5/24 [=====>........................] - ETA: 2:28 - loss: 1.3884 - accuracy: 0.2937

KeyboardInterrupt: 

In [22]:
#!pip install -U scikit-learn

from sklearn.metrics import classification_report, confusion_matrix

test_data_gen = ImageDataGenerator(rescale=1 / 255)

test_set = val_data_gen.flow_from_directory('weather/data_test',
                                                  target_size=target_size,
                                                  batch_size=val_batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 172 images belonging to 4 classes.


In [35]:
filenames = test_set.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_set,steps = nb_samples)
predicted_classes = predict.argmax(axis=-1)


print(classification_report(test_set.classes, predicted_classes))
print(confusion_matrix(test_set.classes, predicted_classes))

C:\Users\Admin\anaconda3\envs\DL_Project\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


              precision    recall  f1-score   support

           0       0.24      0.62      0.35        45
           1       0.00      0.00      0.00        33
           2       0.00      0.00      0.00        39
           3       0.34      0.35      0.34        55

    accuracy                           0.27       172
   macro avg       0.15      0.24      0.17       172
weighted avg       0.17      0.27      0.20       172

[[28  0  0 17]
 [24  0  0  9]
 [28  0  0 11]
 [35  0  1 19]]


C:\Users\Admin\anaconda3\envs\DL_Project\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\envs\DL_Project\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\envs\DL_Project\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model 2

# Model 3